In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/GP/Linnaeus_Esrgan.zip

Streaming output truncated to the last 5000 lines.
  inflating: content/Linnaeus_Esrgan/train/berry/132_128_rlt.png  
 extracting: content/Linnaeus_Esrgan/train/berry/811_128_rlt.png  
  inflating: content/Linnaeus_Esrgan/train/berry/437_128_rlt.png  
  inflating: content/Linnaeus_Esrgan/train/berry/1091_128_rlt.png  
  inflating: content/Linnaeus_Esrgan/train/berry/962_128_rlt.png  
  inflating: content/Linnaeus_Esrgan/train/berry/239_128_rlt.png  
  inflating: content/Linnaeus_Esrgan/train/berry/104_128_rlt.png  
  inflating: content/Linnaeus_Esrgan/train/berry/613_128_rlt.png  
  inflating: content/Linnaeus_Esrgan/train/berry/422_128_rlt.png  
  inflating: content/Linnaeus_Esrgan/train/berry/2_128_rlt.png  
  inflating: content/Linnaeus_Esrgan/train/berry/1068_128_rlt.png  
 extracting: content/Linnaeus_Esrgan/train/berry/1102_128_rlt.png  
  inflating: content/Linnaeus_Esrgan/train/berry/185_128_rlt.png  
  inflating: content/Linnaeus_Esrgan/train/berry/632_128_rlt.png  
  inflatin

In [ ]:
import numpy as np
import pandas as pd
from pandas import Series

import pathlib
from glob import glob
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing import image

from tensorflow import keras
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D,BatchNormalization
from tensorflow.python.keras.models import Sequential, Model
from keras.applications.vgg19 import VGG19
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import layers
from keras.applications.xception import Xception
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, BackupAndRestore
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image
from tensorflow.keras import regularizers

In [ ]:
src_path_train = "/content/content/Linnaeus_Esrgan/train"
src_path_val = "/content/content/Linnaeus_Esrgan/val"
src_path_test = "/content/content/Linnaeus_Esrgan/test"

In [ ]:
import os
os.system("rm -rf /content/content/Linnaeus_Esrgan/train/.ipynb_checkpoints")

0

In [ ]:
#Data Augmentation Function: Let's define an instance of the ImageDataGenerator class and set the parameters.We have to instantiate for the Train,Validation and Test datasets
train_generator = ImageDataGenerator(rescale=1./255,
                                     rotation_range=15,
    zoom_range=0.1,
    horizontal_flip=True)

val_generator = ImageDataGenerator(rescale=1./255,
                                   rotation_range=15,
    zoom_range=0.1,
    horizontal_flip=True)

test_generator = ImageDataGenerator(rescale=1./255)

In [ ]:
#Fit the augmentation method to the data
train_generatorr = train_generator.flow_from_directory(src_path_train,
                            
                                                    class_mode='categorical',
                                                    batch_size=32,
                                                    color_mode="rgb",
                                                    target_size=(128,128),
                                                    shuffle=False,
                                                    seed=42)

validation_generator = train_generator.flow_from_directory(src_path_val,
                                                        target_size=(128,128),
                                                        batch_size=32,
                                                        color_mode="rgb",
                                                        class_mode='categorical',
                                                        shuffle=False,
                                                        seed=42)
test_generatorr = test_generator.flow_from_directory(
    src_path_test,
    target_size=(128,128),
    color_mode="rgb",
    batch_size=1,
    class_mode="categorical",
    shuffle=False,
    seed=42
)

Found 4800 images belonging to 5 classes.
Found 1200 images belonging to 5 classes.
Found 2000 images belonging to 5 classes.


In [ ]:
mobile = tf.keras.applications.mobilenet.MobileNet(
    include_top = False,
    weights="imagenet",
    input_tensor=None,
    input_shape=(128,128,3),
    pooling="max",
    classes=5,
    classifier_activation="softmax")
mobile.summary()

17225924/17225924 [==============================] - 0s 0us/step
Model: "mobilenet_1.00_128"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 64, 64, 32)        864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 64, 64, 32)       128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 64, 64, 32)        0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 64, 64, 32)       288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 64, 64, 32) 

In [ ]:
from keras import Sequential

for layer in mobile.layers:
  layer.trainable = False

model_2=Sequential()
#Add the Dense layers along with activation and batch normalization
model_2.add(mobile)
model_2.add(Flatten())

#Add the Dense layers along with activation and batch normalization
model_2.add(Dense(512,activation=('relu')))
model_2.add(Dropout(.5, seed=42))
model_2.add(Dense(256,activation=('relu')))
#model_2.add(Dense(2048,activation=('relu'))) 
#model_2.add(Dropout(.2, seed=42))
# model_2.add(Dense(2048,activation=('relu'), kernel_regularizer=regularizers.L2(l2=0.01))) 
# model_2.add(Dropout(.3, seed=42))#Adding a dropout layer that will randomly drop 50% of the weights
# model_2.add(Dense(1024,activation=('relu'), kernel_regularizer=regularizers.L2(l2=0.01)))
# model_2.add(Dropout(.2, seed=42))
model_2.add(Dense(5,activation=('softmax'))) #This is the classification layer

model_2.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenet_1.00_128 (Functio  (None, 1024)             3228864   
 nal)                                                            
                                                                 
 flatten (Flatten)           (None, 1024)              0         
                                                                 
 dense (Dense)               (None, 512)               524800    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dense_2 (Dense)             (None, 5)                 1285      
                                                        

In [ ]:
from keras.optimizers import SGD,Adam
# from tensorflow.keras.optimizers.legacy import SGD
learning_rate=.0001

sgd = tf.keras.optimizers.legacy.SGD(lr=learning_rate,momentum=.9,nesterov=False)
adam = Adam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

/usr/local/lib/python3.9/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [ ]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', patience=5, verbose=1, factor=0.1, min_lr=0.00001,min_delta=0.001)
# early_stopping_monitor = EarlyStopping(patience=7, monitor='val_accuracy', restore_best_weights=True)

In [ ]:
model_2.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
history = model_2.fit_generator(
              train_generatorr,
              steps_per_epoch = train_generatorr.samples // 32,
              validation_data = validation_generator, 
              validation_steps = validation_generator.samples // 32,
              callbacks = [learning_rate_reduction],
              epochs = 5)

Epoch 1/5


<ipython-input-31-c2f096938360>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model_2.fit_generator(


150/150 [==============================] - 29s 192ms/step - loss: 0.1725 - accuracy: 0.9354 - val_loss: 0.3399 - val_accuracy: 0.8953 - lr: 1.0000e-04
Epoch 2/5
150/150 [==============================] - 29s 190ms/step - loss: 0.1628 - accuracy: 0.9396 - val_loss: 0.3023 - val_accuracy: 0.9122 - lr: 1.0000e-04
Epoch 3/5
150/150 [==============================] - 29s 190ms/step - loss: 0.1648 - accuracy: 0.9358 - val_loss: 0.3230 - val_accuracy: 0.8953 - lr: 1.0000e-04
Epoch 4/5
150/150 [==============================] - 29s 192ms/step - loss: 0.1594 - accuracy: 0.9404 - val_loss: 0.3428 - val_accuracy: 0.9012 - lr: 1.0000e-04
Epoch 5/5
150/150 [==============================] - 28s 190ms/step - loss: 0.1593 - accuracy: 0.9396 - val_loss: 0.2965 - val_accuracy: 0.9122 - lr: 1.0000e-04


In [ ]:
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
evaluate = model_2.evaluate(test_generatorr)
print(evaluate)

2000/2000 [==============================] - 14s 7ms/step - loss: 0.2469 - accuracy: 0.9280
[0.24693600833415985, 0.9279999732971191]


In [ ]:
model_2.save("ESRGAN_Linnaeus_MobileNet.h5")